<a href="https://colab.research.google.com/github/t8101349/Costco-discounted-goods-crawler-bot/blob/main/costco_linebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install line-bot-sdk flask


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.9/779.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.2 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y curl
!curl -O https://dl.google.com/dl/cloudsdk/channels/rapid/downloads/google-cloud-sdk-430.0.0-linux-x86_64.tar.gz
!tar -xzf google-cloud-sdk-430.0.0-linux-x86_64.tar.gz
!./google-cloud-sdk/install.sh -q
!./google-cloud-sdk/bin/gcloud init

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,793 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,773 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,686

In [3]:
%%writefile linebot.py
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import os
import requests
import json
import time

app = Flask(__name__)

CHANNEL_ACCESS_TOKEN = 'ik9reNYdHWJhkqdey1cIrtKNF/yMKWyCGTZmkfeZH4FtGSj/hwZmHputQN/95c5dW29MRScjKcLwRFwD67vw8pMON62xtn/lbxm/7X7jVHplqvOtFM9jAGb+azrpqSEULEzdr4OSCJj2aAYm2wH21AdB04t89/1O/w1cDnyilFU='
CHANNEL_SECRET = 'a32b70f15b6fe55ec4ae766e1325cd16'

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# Costco 折扣抓取邏輯
api_url = "https://www.costco.com.tw/rest/v2/taiwan/products/search"
params = {
    "fields": "FULL",
    "query": ":BazaarVoiceRating-desc:bazaarVoiceAverageRatingFacet:4-5:bazaarVoiceAverageRatingFacet:5:bazaarVoiceAverageRatingFacet:3-4",
    "pageSize": 3,
    "sort": "BazaarVoiceRating-desc",
    "category": "Coupon",
    "lang": "zh_TW",
    "curr": "TWD",
    "currentPage": 0
}

def fetch_costco_data():
    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json"}
    response = requests.get(api_url, headers=headers, params=params)
    products_info = []

    if response.status_code == 200:
        products = response.json().get('products', [])
        for product in products:
            name = product.get("name")
            price = product.get("price", {'value':"售完"}).get("value")
            currency = product.get("price", {}).get("currencyIso", "")
            rating = product.get("averageRating", "無評價")
            image_url = 'https://www.costco.com.tw' + product.get("images", [{}])[0].get("url", "無圖片")
            products_info.append(f"商品: {name}\n價格: {price} {currency}\n評價: {rating}\n{image_url}\n")
    return products_info

@app.route('/callback', methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature')
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_message = event.message.text
    if user_message == "查詢折扣":
        costco_data = fetch_costco_data()
        reply = "\n".join(costco_data) if costco_data else "暫無優惠資訊"
    else:
        reply = f"你說了：{user_message}"
    line_bot_api.reply_message(event.reply_token, TextSendMessage(text=reply))

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 5000))
    app.run(host="0.0.0.0", port=port, debug=True)

Writing linebot.py


In [4]:
# 驗證 GCP Service Account
from google.colab import files
uploaded = files.upload()  # 上傳 service-account-key.json


Saving costco-line-e1c5befd361d.json to costco-line-e1c5befd361d.json


In [5]:
!gcloud config set project costco-line


Updated property [core/project].


In [6]:
!./google-cloud-sdk/bin/gcloud auth activate-service-account --key-file=/content/costco-line-e1c5befd361d.json

# 設定專案與區域
!./google-cloud-sdk/bin/gcloud config set project costco-line
!./google-cloud-sdk/bin/gcloud config set run/region asia-east1

Activated service account credentials for: [costco-line@costco-line.iam.gserviceaccount.com]
Updated property [core/project].
Updated property [run/region].


In [36]:
%%writefile Dockerfile
# 使用 Python 基礎映像
FROM python:3.11-slim

WORKDIR /app

# 複製程式碼
COPY . .

# 安裝依賴
RUN pip install --no-cache-dir -r requirements.txt

# 設定環境變數
ENV LINE_CHANNEL_ACCESS_TOKEN="ik9reNYdHWJhkqdey1cIrtKNF/yMKWyCGTZmkfeZH4FtGSj/hwZmHputQN/95c5dW29MRScjKcLwRFwD67vw8pMON62xtn/lbxm/7X7jVHplqvOtFM9jAGb+azrpqSEULEzdr4OSCJj2aAYm2wH21AdB04t89/1O/w1cDnyilFU="
ENV LINE_CHANNEL_SECRET="a32b70f15b6fe55ec4ae766e1325cd16"

# 開放 Flask 預設 Port
EXPOSE 5000

CMD ["python", "linebot.py"]

Overwriting Dockerfile


In [48]:
%%writefile .dockerignore
google-cloud-sdk
sample_data
costco-line-e1c5befd361d.json

Overwriting .dockerignore


In [29]:
%%writefile requirements.txt

flask==2.3.3
line-bot-sdk==3.1.0
requests==2.31.0


Overwriting requirements.txt


In [49]:
!pip install docker


In [50]:
!sudo systemctl start docker
!sudo systemctl enable docker  # 保證 Docker 開機啟動


System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [52]:
!docker build -t costco-bot .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

ERRO[0005] Can't add file /content/.config/credentials.db to tar: io: read/write on closed pipe 
ERRO[0005] Can't close tar writer: io: read/write on closed pipe 
Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [20]:
!gsutil mb -l asia-east1 -p costco-line gs://costco-bot

Creating gs://costco-bot/...
ServiceException: 409 A Cloud Storage bucket named 'costco-bot' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [53]:
# 建立映像並推送到 Cloud Run

!./google-cloud-sdk/bin/gcloud builds submit --tag gcr.io/costco-line/line-bot --gcs-log-dir=gs://costco-bot


Creating temporary tarball archive of 24287 file(s) totalling 776.0 MiB before compression.
Uploading tarball of [.] to [gs://costco-line_cloudbuild/source/1743599689.78725-8009f00584a84629b1d63a95359a40e6.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/costco-line/locations/global/builds/ef5ca952-37be-4b20-94e9-b37c6f00352f].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ef5ca952-37be-4b20-94e9-b37c6f00352f?project=498723905289 ].
ERROR: (gcloud.builds.submit) 
The build is running, and logs are being written to the default logs bucket.
This tool can only stream logs if you are Viewer/Owner of the project and, if applicable, allowed by your VPC-SC security policy.

The default logs bucket is always outside any VPC-SC security perimeter.
If you want your logs saved inside your VPC-SC perimeter, use your own bucket.
See https://cloud.google.com/build/docs/securing-builds/store-manage-build-logs.



In [25]:
!./google-cloud-sdk/bin/gcloud run deploy linebot \
    --image gcr.io/costco-line/costco-bot \
    --platform managed \
    --region us-central1 \
    --allow-unauthenticated \
    --set-env-vars LINE_CHANNEL_ACCESS_TOKEN='ik9reNYdHWJhkqdey1cIrtKNF/yMKWyCGTZmkfeZH4FtGSj/hwZmHputQN/95c5dW29MRScjKcLwRFwD67vw8pMON62xtn/lbxm/7X7jVHplqvOtFM9jAGb+azrpqSEULEzdr4OSCJj2aAYm2wH21AdB04t89/1O/w1cDnyilFU=',LINE_CHANNEL_SECRET='a32b70f15b6fe55ec4ae766e1325cd16'



Deploying container to Cloud Run service [linebot] in project [costco-line] region [us-central1]
ERROR: (gcloud.run.deploy) Revision 'linebot-00001-wut' is not ready and cannot serve traffic. Image 'gcr.io/costco-line/costco-bot' not found.
